In [ ]:
# Install our (local, for now) pip package 
!pip3 install -e /home/jjhenkel/mount/velveeta/find-tune

In [ ]:
# Create azure auth stuff (service principal for container operations)
!az ad sp create-for-rbac --sdk-auth > ./.auth/gsl-find-tune-sp.json 

# Setup .auth/.env (these are secret, so nothing included here)

In [1]:
import mlos

mlos.checkenv(using_azure=True)

project = (
  mlos.create_project_from_git_info()
      .with_backend(
          mlos.backends()
              .azure()
      )
      .with_build_options(
          mlos.build_options()
              .compile_with_n_threads(lambda cpu_threads: cpu_threads // 2)
              .override_plugin(
                  '/home/jjhenkel/mount/velveeta/find-tune/frontends/clang/src/libDumpDatalog.so'
              )
      )
)

# Creates backend resources (VMs / nsgs / ips / etc.)
project.allocate(attach=True, just_db=True)

- Verifying environment...
   - Checking for 'POSTGRES_PASSWORD' in environment...
      + Found POSTGRES_PASSWORD:=****************
   - Checking for 'POSTGRES_USER' in environment...
      + Found POSTGRES_USER:=ft-admin
   - Checking for 'POSTGRES_PORT' in environment...
      + Found POSTGRES_PORT:=5432
   - Checking for 'MIDDLEWARE_PORT' in environment...
      + Found MIDDLEWARE_PORT:=8123
   - Checking for 'DOCKER_HUB_USER' in environment...
      + Found DOCKER_HUB_USER:=findtune
   - Checking for 'DOCKER_HUB_PASSWORD' in environment...
      + Found DOCKER_HUB_PASSWORD:=***************
   - Checking for 'AZURE_RESOURCE_GROUP' in environment...
      + Found AZURE_RESOURCE_GROUP:=FindTune-EUS
   - Checking for 'AZURE_AUTH_LOCATION' in environment...
      + Found AZURE_AUTH_LOCATION:=.auth/gsl-find-tune-sp.json
   + Environment verification complete
- Creating project from current git repository...
   - Getting project name: 
      + project-name:='https://github.com/jjhenkel/l

In [ ]:
# Remotely builds + ingests this code for analysis
project.ingest()

In [14]:
project.poll_status()

- Polling for remote build status...
   * Built /app/table/merger.cc @2020-08-04 17:09:40.747614+00:00
   * Built /app/util/arena.cc @2020-08-04 17:10:57.803679+00:00
   * Built /app/util/bloom.cc @2020-08-04 17:11:02.436208+00:00
   * Built /app/table/iterator.cc @2020-08-04 17:11:06.408695+00:00
   * Built /app/util/filter_policy.cc @2020-08-04 17:11:10.800569+00:00
   * Built /app/util/options.cc @2020-08-04 17:11:31.880521+00:00
   * Built /app/util/env.cc @2020-08-04 17:11:36.969249+00:00
   * Built /app/util/coding.cc @2020-08-04 17:11:42.298382+00:00
   * Built /app/table/two_level_iterator.cc @2020-08-04 17:11:49.455263+00:00
   * Built /app/util/status.cc @2020-08-04 17:11:54.209595+00:00
   * Built /app/util/cache.cc @2020-08-04 17:12:00.748802+00:00
   * Built /app/util/crc32c.cc @2020-08-04 17:12:12.827319+00:00
   * Built /app/db/filename.cc @2020-08-04 17:12:17.962587+00:00
   * Built /app/util/hash.cc @2020-08-04 17:12:23.887749+00:00
   * Built /app/table/format.cc @202

In [ ]:
# Ensures backend resources are cleaned up  
project.deallocate() # detach=True (detach just "lets go" and maybe stops VMs, False incurs a delete and maybe prompts user for conf)

In [12]:
import psycopg2
import psycopg2.extras

conn = psycopg2.connect(project.backend.db_conn_str)
cursor = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)


In [13]:
cursor.execute("SELECT CONCAT(CONCAT(project_name, '/'), commit_hash) FROM meta_projects")
for row in cursor.fetchall():
    print(row)

RealDictRow([('concat', 'https://github.com/jjhenkel/leveldb/1454924aacba2ea3cdf183bbed512566a600b05a')])


In [13]:
project.backend.db_conn_str

'host=52.186.99.213 user=ft-admin password=vie0thahnahpheKe'